In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from  tensorflow.keras.preprocessing.sequence import pad_sequences

2023-11-19 15:18:04.009235: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
restaurant_data = pd.read_csv("final_df.csv")

In [3]:
restaurant_data.columns

Index(['name', 'cuisine', 'lat', 'lon', 'rating', 'Has Menu', 'locale',
       'review_objects', 'dishes', 'price', 'type', 'popular_times',
       'in_cuisine', 'japanese', 'chinese', 'bar', 'western', 'indian',
       'mediterranean', 'mexican', 'italian', 'local', 'malaysian', 'hipster',
       'veg', 'fast_food', 'thai', 'korean', 'halal', 'dessert', 'seafood',
       'health', 'french', 'vietnamese', 'michellin'],
      dtype='object')

In [4]:
restaurant_data_clean  = restaurant_data.drop(['cuisine', 'lat', 'lon', 'rating', 'Has Menu', 'locale', 'review_objects', 'dishes', 'type', 'popular_times', 'in_cuisine'], axis = 1)

In [5]:
restaurant_data_clean['price'].unique()

array(['$$', '$', '$$$', '$$ - $$$', '$$$$', '$$$$$'], dtype=object)

In [6]:
restaurant_data_clean.loc[restaurant_data_clean['price'] == '$', 'price'] = 0
restaurant_data_clean.loc[restaurant_data_clean['price'] == '$$', 'price'] = 1
restaurant_data_clean.loc[restaurant_data_clean['price'] == '$$ - $$$', 'price'] = 2
restaurant_data_clean.loc[restaurant_data_clean['price'] == '$$$', 'price'] = 3
restaurant_data_clean.loc[restaurant_data_clean['price'] == '$$$$', 'price'] = 4
restaurant_data_clean.loc[restaurant_data_clean['price'] == '$$$$$', 'price'] = 5

In [7]:
restaurant_name_list = restaurant_data_clean['name']
restaurant_data_clean.drop(['name'], axis = 1, inplace = True)

In [8]:
restaurant_data_clean
## Create df from streamlit

,price,japanese,chinese,bar,western,indian,mediterranean,mexican,italian,local,...,fast_food,thai,korean,halal,dessert,seafood,health,french,vietnamese,michellin
0,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,1,0,0,0,0
2,3,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4199,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4200,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4201,3,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4202,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [9]:
restaurant_data_clean.to_csv('restaurant_data_clean.csv', index = False)

In [10]:
restaurant_numpy = restaurant_data_clean.to_numpy()

In [18]:
# Save restaurant_numpy
np.save('restaurant_numpy.npy', restaurant_numpy)

In [11]:
class MF():

    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [13]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [14]:
print(restaurant_numpy, restaurant_numpy.shape)    

[[1 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [3 0 0 ... 0 0 0]
 ...
 [3 0 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]] (4204, 23)


In [33]:
restaurant_data_clean['price'].unique()

array([1, 0, 3, 2, 4, 5], dtype=object)

In [16]:
# Implement Content Based Filtering 
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm
'''
# Price, Cuisine Not needed anymore
item = np.array([[1, 0, 1],[1, 1, 1], [0, 1, 1], [0, 0, 1]])
user = np.array([[0, 1, 1], [1, 1, 0]])
'''

'\n# Price, Cuisine Not needed anymore\nitem = np.array([[1, 0, 1],[1, 1, 1], [0, 1, 1], [0, 0, 1]])\nuser = np.array([[0, 1, 1], [1, 1, 0]])\n'

In [19]:
# Item
item = restaurant_numpy
# User
rng = np.random.default_rng(12345)
user_cusine = rng.integers(low = 0, high = 2, size = (3,22))
price_user = rng.integers(low= 0, high = 6, size=(3,1))
user = np.concatenate((price_user, user_cusine), axis = 1)
print(user, user.shape)
#print(np.random.rand(3,23))


[[3 1 0 1 0 0 1 1 1 1 0 1 0 1 1 0 0 0 1 1 1 1 0]
 [4 1 1 1 1 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 0]
 [4 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 1 1 0 1]] (3, 23)


In [20]:
# Save user array
np.save('user.npy', user)

In [21]:
def get_item_user(user_arr, item):
    user_item = []
    for user in user_arr:
        cos_sim = dot(user, item.T)/(norm(user)*norm(item))
        user_item.append(cos_sim)

    return user_item

In [22]:
item_user = get_item_user(user, item)
print(len(item_user), len(item_user[0]))
print(["{0:0.2f}".format(i) for i in item_user[0]])

3 4204
['0.01', '0.00', '0.02', '0.00', '0.01', '0.01', '0.00', '0.01', '0.00', '0.00', '0.00', '0.00', '0.01', '0.01', '0.00', '0.00', '0.00', '0.00', '0.01', '0.00', '0.01', '0.00', '0.00', '0.00', '0.00', '0.00', '0.01', '0.00', '0.01', '0.01', '0.02', '0.00', '0.01', '0.00', '0.02', '0.00', '0.00', '0.01', '0.02', '0.01', '0.01', '0.01', '0.01', '0.00', '0.01', '0.01', '0.01', '0.01', '0.00', '0.00', '0.01', '0.00', '0.01', '0.00', '0.00', '0.02', '0.00', '0.00', '0.01', '0.00', '0.01', '0.02', '0.00', '0.00', '0.00', '0.01', '0.00', '0.01', '0.01', '0.01', '0.01', '0.01', '0.01', '0.00', '0.00', '0.01', '0.00', '0.00', '0.02', '0.01', '0.01', '0.01', '0.01', '0.00', '0.01', '0.00', '0.00', '0.00', '0.00', '0.01', '0.02', '0.00', '0.00', '0.01', '0.00', '0.01', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.01', '0.00', '0.00', '0.01', '0.02', '0.00', '0.01', '0.01', '0.01', '0.01', '0.01', '0.00', '0.00', '0.00', '0.00', '0.00', '0.02', '0.02', '0.00', '0.00', '0.01', '0.01', 

In [32]:
from sklearn.preprocessing import  MinMaxScaler

scaler_MinMax = MinMaxScaler()
def scale_item_user(item_user):
    for i in range(0, len(item_user)):
        reshaped_item = item_user[i].reshape(-1,1)
        scaled_item = scaler_MinMax.fit_transform(reshaped_item).reshape(1,-1)[0]
        item_user[i] = scaled_item

    return item_user

item_user = scale_item_user(item_user)


In [30]:
print(item_user[0])

[0.29411765 0.11764706 0.58823529 ... 0.58823529 0.23529412 0.23529412]


In [31]:
print(len(item_user[0]))

4204


In [26]:
index_of_item_user = sorted(range(len(item_user[0])), key=lambda i: item_user[0][i])[-3:]

In [27]:
index_of_item_user

[4044, 4176, 1684]

In [28]:
for index in index_of_item_user:
    print(item_user[0][index])

0.9411764705882353
0.9411764705882353
1.0


In [34]:
for i in range(0, 23):
    print(restaurant_data_clean.columns[i], user[0][i])

price 3
japanese 1
chinese 0
bar 1
western 0
indian 0
mediterranean 1
mexican 1
italian 1
local 1
malaysian 0
hipster 1
veg 0
fast_food 1
thai 1
korean 0
halal 0
dessert 0
seafood 1
health 1
french 1
vietnamese 1
michellin 0


In [35]:
print(restaurant_data_clean.loc[1684,:])
print(restaurant_data_clean.loc[4176,:])
print(restaurant_data_clean.loc[4044,:])

price            5
japanese         0
chinese          0
bar              0
western          0
indian           0
mediterranean    0
mexican          0
italian          0
local            0
malaysian        0
hipster          0
veg              0
fast_food        0
thai             1
korean           0
halal            0
dessert          0
seafood          1
health           0
french           0
vietnamese       0
michellin        0
Name: 1684, dtype: object
price            5
japanese         0
chinese          0
bar              0
western          1
indian           0
mediterranean    0
mexican          0
italian          0
local            0
malaysian        0
hipster          1
veg              1
fast_food        0
thai             0
korean           0
halal            0
dessert          0
seafood          0
health           0
french           0
vietnamese       0
michellin        0
Name: 4176, dtype: object
price            5
japanese         1
chinese          0
bar              

In [36]:
def item_item(item):
    item_item = []
    for i in range(0, len(item)):
        item_similarity = []
        for j in range(0, len(item)):
            #if(i!=j):
            item_i = item[i]
            item_j = item[j]
            cos_sim = dot(item[i], item[j])/(norm(item[i])*norm(item[j]))
            item_similarity.append(cos_sim)
        item_item.append(item_similarity)

    return item_item

item_item_matrix = item_item(item)
    
# Send as csv

In [37]:
# Save item_item_matrix
np.save('item_item_matrix.npy', item_item_matrix)

In [38]:
print(sorted(item_item_matrix[0], reverse = True))

[1.0000000000000002, 0.9428090415820635, 0.9428090415820635, 0.9428090415820635, 0.9428090415820635, 0.8728715609439696, 0.8728715609439696, 0.8728715609439696, 0.8728715609439696, 0.8728715609439696, 0.8660254037844387, 0.8660254037844387, 0.8660254037844387, 0.8660254037844387, 0.8333333333333335, 0.8333333333333335, 0.8333333333333335, 0.8333333333333335, 0.8333333333333335, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259, 0.8164965809277259,

In [39]:
print(item[514])
print(item[0])
print(item_item_matrix[0][514])

[1 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.8660254037844387


In [40]:
index_of_item_item = sorted(range(len(item_item_matrix)), key=lambda i: item_item_matrix[i])[-4:]

In [41]:
for index in index_of_item_item:
    print(index)
    print(item_item_matrix[0][index])
#print(item_item_matrix[index_of_item_item])

2387
0.9428090415820635
2724
0.9428090415820635
3028
0.9428090415820635
0
1.0000000000000002


In [42]:
print(restaurant_name_list[index_of_item_item] , restaurant_data_clean.loc[index_of_item_item, :])

2387    momoya japanese restaurant
2724                    pastaholic
3028                       sg taps
0                  joy dining hall
Name: name, dtype: object      price  japanese  chinese  bar  western  indian  mediterranean  mexican  \
2387     2         1        0    0        0       0              0        0   
2724     2         1        0    0        0       0              0        0   
3028     2         1        0    0        0       0              0        0   
0        1         1        0    0        0       0              0        0   

      italian  local  ...  fast_food  thai  korean  halal  dessert  seafood  \
2387        1      0  ...          0     0       0      0        0        0   
2724        1      0  ...          0     0       0      0        0        0   
3028        1      0  ...          0     0       0      0        0        0   
0           1      0  ...          0     0       0      0        0        0   

      health  french  vietnamese  michelli

In [43]:
print(restaurant_data_clean.loc[2724, :])
print(restaurant_data_clean.loc[3028, :])
print(restaurant_data_clean.loc[0, :])

price            2
japanese         1
chinese          0
bar              0
western          0
indian           0
mediterranean    0
mexican          0
italian          1
local            0
malaysian        0
hipster          0
veg              0
fast_food        0
thai             0
korean           0
halal            0
dessert          0
seafood          0
health           0
french           0
vietnamese       0
michellin        0
Name: 2724, dtype: object
price            2
japanese         1
chinese          0
bar              0
western          0
indian           0
mediterranean    0
mexican          0
italian          1
local            0
malaysian        0
hipster          0
veg              0
fast_food        0
thai             0
korean           0
halal            0
dessert          0
seafood          0
health           0
french           0
vietnamese       0
michellin        0
Name: 3028, dtype: object
price            1
japanese         1
chinese          0
bar              

In [44]:


restaurant_data_clean[(restaurant_data_clean['price'] == 1) & (restaurant_data_clean['japanese'] == 1) & restaurant_data_clean['italian'] == 1]


,price,japanese,chinese,bar,western,indian,mediterranean,mexican,italian,local,...,fast_food,thai,korean,halal,dessert,seafood,health,french,vietnamese,michellin
0,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
514,1,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1540,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2385,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2465,1,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2747,1,1,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0


In [45]:
print(restaurant_name_list[514])
print(restaurant_data_clean.loc[514, :])

bold bowls
price            1
japanese         1
chinese          0
bar              0
western          1
indian           0
mediterranean    0
mexican          0
italian          1
local            0
malaysian        0
hipster          0
veg              0
fast_food        0
thai             0
korean           0
halal            0
dessert          0
seafood          0
health           0
french           0
vietnamese       0
michellin        0
Name: 514, dtype: object


In [46]:
print(item_item_matrix[1684])

print(np.shape(item_item_matrix))

[0.5555555555555556, 0.09622504486493763, 0.8703882797784892, 0.0, 0.8606629658238704, 0.4811252243246881, 0.1360827634879543, 0.5555555555555556, 0.0, 0.0, 0.0, 0.0, 0.8606629658238704, 0.6804138174397716, 0.0, 0.0, 0.0, 0.0, 0.6666666666666666, 0.0, 0.6804138174397716, 0.1360827634879543, 0.0, 0.0, 0.0, 0.0, 0.5773502691896257, 0.0, 0.5555555555555556, 0.727392967453308, 0.9737289911202951, 0.0, 0.6666666666666666, 0.0, 0.8703882797784892, 0.0, 0.0, 0.6804138174397716, 0.9128709291752767, 0.8606629658238704, 0.727392967453308, 0.6804138174397716, 0.8001322641986387, 0.0, 0.6666666666666666, 0.5555555555555556, 0.727392967453308, 0.5555555555555556, 0.0, 0.0, 0.8001322641986387, 0.0, 0.6804138174397716, 0.0, 0.0, 0.8888888888888888, 0.0, 0.0, 0.6666666666666666, 0.0, 0.5773502691896257, 0.9737289911202951, 0.0, 0.0, 0.1360827634879543, 0.5555555555555556, 0.0, 0.4811252243246881, 0.5555555555555556, 0.6804138174397716, 0.727392967453308, 0.7856742013183862, 0.7856742013183862, 0.0, 0.

In [47]:
# Collaborative 

#print(len(restaurant_data_clean))

user_swiped = [[-1 , 0, 1, 1], [1 ,0, 0, 1]]
user_swiped = rng.integers(low = 0, high = 2, size = (3, len(restaurant_data_clean)))
print(user_swiped.shape)
R = user_swiped
print(R.shape)
mf = MF(R, K=2, alpha=0.1, beta=0.01, iterations=20)

(3, 4204)
(3, 4204)


In [48]:
print(list(user_swiped[0]))

[1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 

In [49]:
training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
print()
print("Global bias:")
print(mf.b)
print()
print("User bias:")
print(mf.b_u)
print()
print("Item bias:")
print(mf.b_i)

Iteration: 10 ; error = 0.0431
Iteration: 20 ; error = 0.0110

P x Q:
[[0.99996594 1.00001999 0.99996149 ... 0.99997485 0.99998463 1.00000946]
 [0.9999726  1.00000959 0.99995764 ... 0.99994805 0.99997168 1.00002405]
 [1.00003687 1.00001033 1.00001199 ... 1.00000912 1.00004116 1.00000689]]

Global bias:
1.0

User bias:
[5.63866820e-06 6.11317159e-06 2.82779054e-05]

Item bias:
[ 5.72121386e-05 -2.89285094e-04 -2.68006380e-04 ... -1.02778650e-03
 -5.82831023e-04  5.68258372e-04]


In [50]:
"""
user_user: [user_1: [item_1, item_2, item_3, item_4], user_2: [item1, item_2, item_3, item_4]
"""
user_user = sigmoid(mf.full_matrix())
sorted(user_user[0], reverse = False)

[0.7302536652050431,
 0.7306216297541593,
 0.7306487455052628,
 0.730782913164645,
 0.7307879665025006,
 0.7307949259861569,
 0.7308036841776075,
 0.7308073820589486,
 0.7308184243202264,
 0.7308273018865831,
 0.7308397439614981,
 0.7308424346193422,
 0.73084274873732,
 0.7308509544661917,
 0.7308562156529615,
 0.7308570341774687,
 0.7308662103028578,
 0.7308687040439398,
 0.7308694794819014,
 0.7308705720433535,
 0.73087193708767,
 0.7308741008195754,
 0.7308743882709516,
 0.7308744313898571,
 0.7308776577099761,
 0.7308785733264132,
 0.7308809622348029,
 0.7308839346584608,
 0.7308869689951936,
 0.7308889464630027,
 0.7308897921335552,
 0.7308914853205724,
 0.7308920639957345,
 0.7308923607904438,
 0.7308934644407157,
 0.73089354380785,
 0.7308946674757024,
 0.7308955316446302,
 0.7308978769940163,
 0.7308981787903918,
 0.730898588395374,
 0.7308993877662303,
 0.7308996328497909,
 0.7308996990514262,
 0.7309002368456036,
 0.7309004959058214,
 0.7309027791051972,
 0.73090395013954,
 0

In [51]:
'''
item_item: [item1: [item_1, item_2, item_3, item_4], item_2: [item_1, item_2, item_3, item_4] ...]
'''
item_item_matrix[0]

[1.0000000000000002,
 0.0,
 0.6963106238227914,
 0.0,
 0.5163977794943222,
 0.5773502691896258,
 0.0,
 0.6666666666666667,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5163977794943222,
 0.40824829046386296,
 0.0,
 0.0,
 0.0,
 0.0,
 0.33333333333333337,
 0.0,
 0.40824829046386296,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2886751345948129,
 0.0,
 0.33333333333333337,
 0.4364357804719848,
 0.5477225575051661,
 0.0,
 0.33333333333333337,
 0.0,
 0.5222329678670935,
 0.0,
 0.0,
 0.40824829046386296,
 0.5477225575051661,
 0.5163977794943222,
 0.8728715609439696,
 0.40824829046386296,
 0.4364357804719848,
 0.0,
 0.33333333333333337,
 0.33333333333333337,
 0.6546536707079772,
 0.33333333333333337,
 0.0,
 0.0,
 0.4364357804719848,
 0.0,
 0.40824829046386296,
 0.0,
 0.0,
 0.5000000000000001,
 0.0,
 0.0,
 0.33333333333333337,
 0.0,
 0.2886751345948129,
 0.5477225575051661,
 0.0,
 0.0,
 0.0,
 0.6666666666666667,
 0.0,
 0.2886751345948129,
 0.6666666666666667,
 0.40824829046386296,
 0.6546536707079772,
 0.47140452079103173,

In [52]:
'''
item_user: [user_1: [item_1, item_2, item_3, item_4], user_2:[ item_1, item_2, item_3, item_4]]
'''
item_user[0]

array([0.29411765, 0.11764706, 0.58823529, ..., 0.58823529, 0.23529412,
       0.23529412])

In [53]:
for i in range(0, len(item_user[0])):
    if(item_user[0][i] == 0.5):
        print(item_user[0][i], user_user[0][i])

In [54]:
def get_collab_content(item_user_arr, user_user_arr):
    collab_content = np.array(item_user_arr) + np.array(user_user_arr)
    return collab_content

In [55]:
collab_content = get_collab_content(item_user, user_user).tolist()
collab_content[0]
sorted(collab_content[0], reverse= True)

[1.7310594078650157,
 1.6722410067185867,
 1.6722367709590489,
 1.672232897878064,
 1.6722285503999323,
 1.6722186429094315,
 1.554594938885324,
 1.55459223572262,
 1.5545900349883466,
 1.5545899594136854,
 1.5545870142071399,
 1.5545857211784377,
 1.5545845254793436,
 1.5545833570529697,
 1.554582085342553,
 1.554576176383471,
 1.4957750932293532,
 1.495769899541842,
 1.4957698144564688,
 1.4957686768992307,
 1.4957672815871832,
 1.495766851566393,
 1.4957665989775724,
 1.4957660346026553,
 1.4957651131231648,
 1.4957636247176103,
 1.495762127502722,
 1.4957621131070953,
 1.4957619038078027,
 1.4957605546627972,
 1.495760419555975,
 1.495760000102544,
 1.4957594303995783,
 1.4957588249762206,
 1.495758750537071,
 1.4957535617614652,
 1.4957535604481078,
 1.4957504615796196,
 1.4957404529660874,
 1.4955973676735135,
 1.4370425154029292,
 1.4369750466648554,
 1.4369517895063244,
 1.4369480603104006,
 1.4369477836859068,
 1.436947762961673,
 1.4369463432461642,
 1.4369456009600081,
 1.43

In [57]:
def get_recommendation(user_index, item_item_matrix, collab_content):
    user_arr = collab_content[user_index]
    recommendation = user_arr.index(max(user_arr))
    print(recommendation)
    n_recommendation = sorted(range(len(item_item_matrix[recommendation])), key=lambda i: item_item_matrix[recommendation][i], reverse = True)[:5]
    #print(item_item_matrix[recommendation])
    #print(sorted(item_item_matrix[recommendation], reverse=True))
    print(n_recommendation)
    return n_recommendation

In [58]:
recco = get_recommendation(0, item_item_matrix, collab_content)

1684
[1684, 1875, 1876, 2486, 2571]


In [59]:
print(user[0])

[3 1 0 1 0 0 1 1 1 1 0 1 0 1 1 0 0 0 1 1 1 1 0]


In [60]:
sample_item = item[1684]
sample_user = user[0]
print(sample_item)
print(sample_user)
#print(sigmoid(dot(sample_user, sample_item.T)))

[5 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0]
[3 1 0 1 0 0 1 1 1 1 0 1 0 1 1 0 0 0 1 1 1 1 0]


In [61]:
print(sorted(item_item_matrix[1684], reverse = True))

[1.0, 0.9864400504156211, 0.9864400504156211, 0.9864400504156211, 0.9864400504156211, 0.9864400504156211, 0.9864400504156211, 0.9864400504156211, 0.9801960588196069, 0.9801960588196069, 0.9801960588196069, 0.9801960588196069, 0.9737289911202951, 0.9737289911202951, 0.9737289911202951, 0.9737289911202951, 0.9737289911202951, 0.9737289911202951, 0.9737289911202951, 0.9737289911202951, 0.9737289911202951, 0.9737289911202951, 0.9737289911202951, 0.9737289911202951, 0.9737289911202951, 0.9525793444156804, 0.9525793444156804, 0.9525793444156804, 0.9467292624062573, 0.9467292624062573, 0.9467292624062573, 0.9467292624062573, 0.9467292624062573, 0.9467292624062573, 0.9467292624062573, 0.9467292624062573, 0.9467292624062573, 0.9444444444444444, 0.9444444444444444, 0.9444444444444444, 0.9435641951204965, 0.9428090415820634, 0.9428090415820634, 0.9428090415820634, 0.9428090415820634, 0.9428090415820634, 0.9428090415820634, 0.9428090415820634, 0.9428090415820634, 0.9335200560186733, 0.933520056018

In [62]:
# User Cuisine
#print(user[0])
# User Swipe
# Restaurant Cuisne
default_recc = recco[0]
for recc in recco:
    print("restaurant name:", restaurant_name_list[recc])
    print("item-item:", item_item_matrix[default_recc][recc])
    print("user swiped:", user_swiped[0][recc], "item-user-similarity:", item_user[0][recc] ,"user-user-similarity:", user_user[0][recc])
    print("collab_content", collab_content[0][recc])
    for i in range(0, 23):
        print(restaurant_data_clean.columns[i],"u:", user[0][i], "r:", restaurant_data_clean.iloc[recc, i])

restaurant name: ja kin mukata
item-item: 1.0
user swiped: 1 item-user-similarity: 1.0 user-user-similarity: 0.7310594078650158
collab_content 1.7310594078650157
price u: 3 r: 5
japanese u: 1 r: 0
chinese u: 0 r: 0
bar u: 1 r: 0
western u: 0 r: 0
indian u: 0 r: 0
mediterranean u: 1 r: 0
mexican u: 1 r: 0
italian u: 1 r: 0
local u: 1 r: 0
malaysian u: 0 r: 0
hipster u: 1 r: 0
veg u: 0 r: 0
fast_food u: 1 r: 0
thai u: 1 r: 1
korean u: 0 r: 0
halal u: 0 r: 0
dessert u: 0 r: 0
seafood u: 1 r: 1
health u: 1 r: 0
french u: 1 r: 0
vietnamese u: 1 r: 0
michellin u: 0 r: 0
restaurant name: khao hom by rung mama
item-item: 0.9864400504156211
user swiped: 1 item-user-similarity: 0.6470588235294117 user-user-similarity: 0.7310586436695684
collab_content 1.37811746719898
price u: 3 r: 3
japanese u: 1 r: 0
chinese u: 0 r: 0
bar u: 1 r: 0
western u: 0 r: 0
indian u: 0 r: 0
mediterranean u: 1 r: 0
mexican u: 1 r: 0
italian u: 1 r: 0
local u: 1 r: 0
malaysian u: 0 r: 0
hipster u: 1 r: 0
veg u: 0 r: 0
f

## Miscallaneous from here on

In [255]:
df_grab = pd.read_csv("complete_grab.csv")

In [43]:
import json
from ast import literal_eval

df_grab

df_tran = pd.DataFrame({})
ind = 0
for cuisine_arr in df_grab['cuisine']:
    i = 1
    dict_foodie = {}
    
    if(type(cuisine_arr) != float):
        cuisine_arr = literal_eval(cuisine_arr)
        for cuisine in cuisine_arr:
            string_iter = 'cuisine'+str(i)
            dict_foodie[string_iter] = 1
            i=i+1
    df_foodie = pd.DataFrame.from_records(dict_foodie, index = [ind])
    df_foodie = pd.concat([df_grab, df_foodie])
    ind+=1

In [21]:
df_foodie

,cuisine1,cuisine10,cuisine11,cuisine12,cuisine13,cuisine14,cuisine15,cuisine16,cuisine17,cuisine18,...,cuisine42,cuisine43,cuisine44,cuisine45,cuisine46,cuisine5,cuisine6,cuisine7,cuisine8,cuisine9
9661,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
